# Classification over Jung's cognitive function types

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandasql as sql

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.semi_supervised import SelfTrainingClassifier
from pandasql import sqldf

In [3]:
db_f_activities = pd.read_pickle('db_exploitable.pkl')
display(db_f_activities)

,report_title,channels,hand_preference,mbti_type,analysis_version,frequency_bandwidths,executive_style,top_patterns,top_skills,average_circuit_hits_decision_making,...,average_circuit_hits_abstract,average_circuit_hits_concrete,average_circuit_hits_social,average_circuit_hits_technical,region_hits_circuit_hits,region_hits_regions,top_circuit_hits_hits,top_circuit_hits_circuits,top_patterns_encoded,top_skills_cleaned
0,10 Daily Activities,"[""CZ"",""FZ"",""FPZ"",""FP1"",""F7"",""F3"",""FC1"",""C3"",""F...",RIGHT,INTP,2001-02-01 00:00:00,"[""0.5:45""]",ENERGIZER,"[""STRONG_STARBURST"",""LEFT"",""CENTER_BACK""]","[""O1"",""PO7"",""P8"",""O2"",""CP6"",""PO8""]",53371,...,55753.57,56380,55519.168,55362.168,"[57007,57152,56292,56591,57018,56339,56678,569...","[""C3"",""CP1"",""O1"",""O2"",""P4"",""PO8"",""C4"",""CP2"",""C...","[4585,1277,982,918,748,616,605,595,564,549,473...","[""C3:C4:CP1:CP2:CP5:CP6:CZ:F3:F4:F7:F8:FC1:FC2...",CB LL SU,CP6 O1 O2 P8 PO7 PO8
1,Daily activities,"[""CZ"",""FZ"",""FPZ"",""FP1"",""F7"",""F3"",""FC1"",""C3"",""F...",LEFT,None,2001-02-01 00:00:00,"[""0.5:45""]",ENERGIZER,"[""STRONG_STARBURST"",""LEFT"",""HALO""]","[""O1"",""PO7"",""P7"",""FT9"",""T7"",""O2""]",49629,...,54468.57,56234.855,55770,57082.832,"[53913,54661,49629,51002,55692,56936,57481,567...","[""F4"",""F8"",""FP1"",""FPZ"",""O1"",""P4"",""PO7"",""PO8"",""...","[1561,694,685,647,624,574,539,519,511,492,485,...","[""O1:PO7"",""O1:P7:PO7"",""FT10:FT9"",""P7:T7"",""P7:P...",HH LL SU,FT9 O1 O2 P7 PO7 T7
2,9 daily activities,"[""CZ"",""FZ"",""FPZ"",""FP1"",""F7"",""F3"",""FC1"",""C3"",""F...",RIGHT,INTP,2001-02-01 00:00:00,"[""0.5:45""]",REFINER,"[""CENTER_FRONT"",""CENTER_BACK"",""RIGHT""]","[""T8"",""PO8"",""P8"",""P7"",""O2"",""T7""]",37356,...,47302.715,47486.285,47424.5,48528.668,"[47755,48280,47823,50311,46119,37356,49120,494...","[""C4"",""CZ"",""FC2"",""P4"",""F8"",""FP1"",""O1"",""PO7"",""P...","[1689,640,630,552,547,531,512,509,493,475,464,...","[""O2:PO8"",""CP6:P8"",""P8:PO8"",""CP5:P7"",""FC6:T8"",...",CB CF RR,O2 P7 P8 PO8 T7 T8
3,10 daily activities,"[""CZ"",""FZ"",""FPZ"",""FP1"",""F7"",""F3"",""FC1"",""C3"",""F...",RIGHT,None,2001-02-01 00:00:00,"[""0.5:45""]",REFINER,"[""BACK"",""LEFT"",""CENTER_BACK""]","[""P7"",""PO7"",""P8"",""PO8"",""O1"",""FT9""]",35679,...,36529.57,36580.855,36142,36649.332,"[37513,36812,37166,36201,36236,31689,33415,363...","[""C4"",""FC2"",""P4"",""FT9"",""O1"",""P7"",""P8"",""PO7"",""C...","[2658,1991,1641,1106,1075,682,513,508,504,504,...","[""P7:PO7"",""P8:PO8"",""O1:PO7"",""O1:P7:PO7"",""P7:P8...",BB CB LL,FT9 O1 P7 P8 PO7 PO8
4,10 Daily Activities,"[""CZ"",""FZ"",""FPZ"",""FP1"",""F7"",""F3"",""FC1"",""C3"",""F...",RIGHT,INFJ,2001-02-01 00:00:00,"[""0.5:45""]",REFINER,"[""BACK"",""HALO"",""HIDDEN_STARBURST""]","[""O1"",""PO7"",""O2"",""PO8"",""P7"",""FT9""]",69148,...,72367.86,73119.14,70975.164,72367.836,"[71292,68829,74352,72801,74012,73500,67784,731...","[""F7"",""O2"",""CP6"",""F3"",""FC1"",""FC5"",""FT9"",""FZ"",""...","[3680,1801,1665,1300,1041,861,776,707,668,650,...","[""O1:PO7"",""O2:PO8"",""O1:O2"",""P7:PO7"",""O1:P7:PO7...",BB DU HH,FT9 O1 O2 P7 PO7 PO8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,10 Daily Activities,"[""CZ"",""FZ"",""FPZ"",""FP1"",""F7"",""F3"",""FC1"",""C3"",""F...",RIGHT,ENFP,2001-02-01 00:00:00,"[""0.5:45""]",ENERGIZER,"[""STRONG_STARBURST"",""RANDOM_FIELD"",""AUDIO_KINE...","[""T7"",""T8"",""O2"",""FT9"",""P7"",""PO8""]",47347,...,49532.715,50133.145,48627.832,48541.168,"[49553,49877,49585,49912,51480,50945,50830,495...","[""CP2"",""CZ"",""FC2"",""C4"",""CP5"",""CP6"",""FC5"",""FT10...","[1531,1226,1064,736,680,672,648,610,565,550,51...","[""T7:T8"",""FT9:T7"",""O2:PO8"",""O2:T7"",""O2:T8"",""CP...",AK NN SU,FT9 O2 P7 PO8 T7 T8
123,10 Daily Activities,"[""CZ"",""FZ"",""FPZ"",""FP1"",""F7"",""F3"",""FC1"",""C3"",""F...",RIGHT,INTJ,2001-02-01 00:00:00,"[""0.5:45""]",REFINER,"[""LEFT"",""HALO"",""CENTER_FRONT""]","[""FT9"","

In [4]:
# Step 1: Clean text columns
def clean_text_column(col):
    return col.apply(lambda x: ' '.join(x) if isinstance(x, list) else (str(x) if x is not None else '')).replace('', 'unknown')

db_f_activities['top_patterns'] = clean_text_column(db_f_activities['top_patterns'])
db_f_activities['top_skills'] = clean_text_column(db_f_activities['top_skills'])

# Step 2: Fix corrupted numeric columns (e.g., strings/dates in numeric columns)
# Convert to numeric safely
numeric_cols = [
    'average_circuit_hits_decision_making', 'average_circuit_hits_perception',
    'average_circuit_hits_abstract', 'average_circuit_hits_concrete',
    'average_circuit_hits_social', 'average_circuit_hits_technical'
]

for col in numeric_cols:
    db_f_activities[col] = pd.to_numeric(db_f_activities[col], errors='coerce')

# Step 3: Remove rows with NaN in any of the numeric columns
query = """
SELECT *
FROM db_f_activities
WHERE 
    average_circuit_hits_abstract IS NOT NULL AND
    average_circuit_hits_concrete IS NOT NULL AND
    average_circuit_hits_social IS NOT NULL AND
    average_circuit_hits_technical IS NOT NULL AND
    average_circuit_hits_decision_making IS NOT NULL AND
    average_circuit_hits_perception IS NOT NULL
"""
db_f_activities_clean = sqldf(query)

In [5]:
# Step 4: Separate labeled and unlabeled
labeled_df = db_f_activities_clean[db_f_activities_clean['mbti_type'].notnull()]
unlabeled_df = db_f_activities_clean[db_f_activities_clean['mbti_type'].isnull()]

# Step 5: Define preprocessing pipelines
text_pipeline = TfidfVectorizer()
num_pipeline = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())


# Definition of types

In [6]:
decision_functions = {
    'INTP': 'Ti',
    'ENTP': 'Ti',
    'INTJ': 'Te',
    'ENTJ': 'Te',
    'ISTP': 'Ti',
    'ESTP': 'Ti',
    'ISTJ': 'Te',
    'ESTJ': 'Te',
    'INFP': 'Fi',
    'ENFP': 'Fi',
    'INFJ': 'Fe', 
    'ENFJ': 'Fe',
    'ISFP': 'Fi',
    'ESFP': 'Fi',
    'ISFJ': 'Fe',
    'ESFJ': 'Fe'
}

analysis_functions = {
    'INTP': 'Ne',
    'ENTP': 'Ne',
    'INTJ': 'Ni',
    'ENTJ': 'Ni',
    'ISTP': 'Se',
    'ESTP': 'Se',
    'ISTJ': 'Si',
    'ESTJ': 'Si',
    'INFP': 'Ne',
    'ENFP': 'Ne',
    'INFJ': 'Ni', 
    'ENFJ': 'Ni',
    'ISFP': 'Se',
    'ESFP': 'Se',
    'ISFJ': 'Si',
    'ESFJ': 'Si'
}

dominant_functions = {
    'INTP': 'Ti',
    'ENTP': 'Ne',
    'INTJ': 'Ni',
    'ENTJ': 'Te',
    'ISTP': 'Ti',
    'ESTP': 'Se',
    'ISTJ': 'Si',
    'ESTJ': 'Te',
    'INFP': 'Fi',
    'ENFP': 'Ne',
    'INFJ': 'Ni', 
    'ENFJ': 'Fe',
    'ISFP': 'Fi',
    'ESFP': 'Se',
    'ISFJ': 'Si',
    'ESFJ': 'Fe'
}

# Decisional type

In [7]:
# Extract decision function label from MBTI type
labeled_df['decision'] = labeled_df['mbti_type'].map(decision_functions)
labeled_df = labeled_df.dropna(subset=['decision'])

# Features and labels
X = labeled_df.drop(columns=['mbti_type', 'decision'])
y = labeled_df['decision']

# Get numeric columns
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numeric_cols)
    ],
    remainder='drop'
)

# Classifier pipeline
clf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf_pipeline.fit(X_train, y_train)
    pred = clf_pipeline.predict(X_test)
    
    y_true.append(y_test.values[0])
    y_pred.append(pred[0])

# Evaluation
print("Classification Report :")
print(classification_report(y_true, y_pred))

print("Confusion Matrix :")
print(confusion_matrix(y_true, y_pred))

# That's not good wow

/var/folders/88/zqbc622x5tb8f2bkxq6dd4h00000gn/T/ipykernel_11383/3762298556.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_df['decision'] = labeled_df['mbti_type'].map(decision_functions)


Classification Report :
              precision    recall  f1-score   support

          Fe       0.29      0.20      0.24        20
          Fi       0.00      0.00      0.00        16
          Te       0.17      0.16      0.17        25
          Ti       0.25      0.38      0.30        21

    accuracy                           0.20        82
   macro avg       0.18      0.19      0.18        82
weighted avg       0.19      0.20      0.19        82

Confusion Matrix :
[[ 4  2  7  7]
 [ 3  0  7  6]
 [ 4  6  4 11]
 [ 3  5  5  8]]


# Analytic type

In [8]:
# Extract decision function label from MBTI type
labeled_df['analysis'] = labeled_df['mbti_type'].map(analysis_functions)
labeled_df = labeled_df.dropna(subset=['analysis'])

# Features and labels
X = labeled_df.drop(columns=['mbti_type', 'analysis'])
y = labeled_df['analysis']

# Get numeric columns
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numeric_cols)
    ],
    remainder='drop'
)

# Classifier pipeline
clf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf_pipeline.fit(X_train, y_train)
    pred = clf_pipeline.predict(X_test)
    
    y_true.append(y_test.values[0])
    y_pred.append(pred[0])

# Evaluation
print("Classification Report :")
print(classification_report(y_true, y_pred))

print("Confusion Matrix :")
print(confusion_matrix(y_true, y_pred))

#HOW ????

Classification Report :
              precision    recall  f1-score   support

          Ne       0.19      0.26      0.22        27
          Ni       0.38      0.38      0.38        32
          Se       0.00      0.00      0.00        10
          Si       0.00      0.00      0.00        13

    accuracy                           0.23        82
   macro avg       0.14      0.16      0.15        82
weighted avg       0.21      0.23      0.22        82

Confusion Matrix :
[[ 7 14  4  2]
 [14 12  3  3]
 [ 7  2  0  1]
 [ 8  4  1  0]]


# Dominant type

In [9]:
# Extract decision function label from MBTI type
labeled_df['dominant'] = labeled_df['mbti_type'].map(dominant_functions)
labeled_df = labeled_df.dropna(subset=['dominant'])

# Features and labels
X = labeled_df.drop(columns=['mbti_type', 'dominant'])
y = labeled_df['dominant']

# Get numeric columns
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numeric_cols)
    ],
    remainder='drop'
)

# Classifier pipeline
clf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf_pipeline.fit(X_train, y_train)
    pred = clf_pipeline.predict(X_test)
    
    y_true.append(y_test.values[0])
    y_pred.append(pred[0])

# Evaluation
print("Classification Report :")
print(classification_report(y_true, y_pred))

print("Confusion Matrix :")
print(confusion_matrix(y_true, y_pred))

Classification Report :
              precision    recall  f1-score   support

          Fe       0.00      0.00      0.00         7
          Fi       0.00      0.00      0.00         5
          Ne       0.13      0.14      0.14        14
          Ni       0.33      0.40      0.36        20
          Se       0.33      0.17      0.22         6
          Si       0.00      0.00      0.00         6
          Te       0.00      0.00      0.00        12
          Ti       0.06      0.08      0.07        12

    accuracy                           0.15        82
   macro avg       0.11      0.10      0.10        82
weighted avg       0.14      0.15      0.14        82

Confusion Matrix :
[[0 0 0 3 0 1 2 1]
 [0 0 1 2 0 0 1 1]
 [1 0 2 4 0 1 1 5]
 [1 1 2 8 1 2 1 4]
 [0 0 1 0 1 1 2 1]
 [2 1 1 0 0 0 1 1]
 [1 1 4 3 0 0 0 3]
 [0 1 4 4 1 0 1 1]]
